In [8]:
from src.shared import convert_data_into_pandas, get_data_from_api, gen_charging_data_df
import pandas as pd

In [12]:
raw_data = get_data_from_api()
raw_df  = pd.json_normalize(raw_data["chargingStations"])

raw_df.head(5)

,id,name,address,description,totalConnectors,amenities,location.lat,location.lng,connectionsTypes.CCS,connectionsTypes.AC Type 2,connectionsTypes.CHAdeMO
0,B8S7YSLRUBVUO,Agerbæk Børnehus,Debelvej 25,Parkering ved børnehuset.,4,[],55.603490,8.811343,[],"[{'id': 'AGB1', 'status': 'AVAILABLE', 'effect...",[]
1,B9OXSNAF20Z5S,Agerbæk Torv,Fåborgvej 177,None,4,"[CAFE, SHOPPING]",55.600484,8.802200,"[{'id': 'ABT1-A', 'status': 'AVAILABLE', 'effe...",[],[]
2,Alnasenter,Alna Senter,Strømsveien 245,Lynladere i 1 etg utendørs parkeringshus ved M...,17,"[RESTROOM, CAFE, SHOPPING]",59.926581,10.849327,"[{'id': 'AL1-1', 'status': 'AVAILABLE', 'effec...","[{'id': 'AL10', 'status': 'AVAILABLE', 'effect...","[{'id': 'AL1-2', 'status': 'AVAILABLE', 'effec..."
3,B8GEWDTNZ8LXC,Alstad Kro og Camping,Alstad 1,None,8,"[RESTROOM, CAFE, REST_STOP, ACCOMMODATION, RES...",68.272286,13.936212,"[{'id': 'ALC1-1', 'status': 'AVAILABLE', 'effe...","[{'id': 'ALC3-1', 'status': 'AVAILABLE', 'effe...","[{'id': 'ALC1-2', 'status': 'AVAILABLE', 'effe..."
4,BA3I6G2IEG4CG,Altdorf,Prackenfelser Str. 8,None,8,"[RESTROOM, CAFE, RESTAURANT, SHOPPING, WIFI, C...",49.381735,11.344057,"[{'id': 'ALT1-A', 'status': 'AVAILABLE', 'effe...",[],[]


In [30]:
charging_data = gen_charging_data_df(raw_df)

In [57]:
# Different tarriff definitions
#  - kr 5,39/kWh
#  - kr 5,39 / kWh
#  - kr 4,29 per kWh
#  - EUR 0.55 per kWh
#  - 5,49 SEK per kWh
#  - 4,29 kr/kWh + 0 kr/min
#  - kr 5,99/kWh. Over 80%: + kr 1/minutt

conn_cols = [c for c in raw_df.columns if c.startswith("connectionsTypes.")]

tarrif_types = []
for _, row in raw_df.iterrows():
    station_id = row["id"]

    for col in conn_cols:
        conn_type = col.split(".", 1)[1]  # extract column type

        connectors = row[col] or []  # each is a list of dicts

        for item in connectors:
            tarrif_types.append(
                {
                    "station_id": station_id,
                    "id": item["id"],
                    "connection_type": conn_type,
                    "tarrif_types": item["tariffDefinition"]
                }
            )

tarrif_df = pd.DataFrame(tarrif_types)
tarrif_df.head(5)

,station_id,id,connection_type,tarrif_types
0,B8S7YSLRUBVUO,AGB1,AC Type 2,DKK 2.69 per kWh
1,B8S7YSLRUBVUO,AGB2,AC Type 2,DKK 2.69 per kWh
2,B8S7YSLRUBVUO,AGB3,AC Type 2,DKK 2.69 per kWh
3,B8S7YSLRUBVUO,AGB4,AC Type 2,DKK 2.69 per kWh
4,B9OXSNAF20Z5S,ABT1-A,CCS,"DKK 2,99 per kWh"


In [58]:
# In Norway, MVA = Merverdiavgift, which means Value-Added Tax (VAT).

# It is a 25% tax added to most goods and services.
# On EV charging invoices or tariffs, “inkl. MVA” means VAT included.
# “eks. MVA” means VAT excluded.

# Example:
# 5,39 kr/kWh inkl. MVA → final price.
# 5,39 kr/kWh eks. MVA → the station will add +25%.

# Lading for Brim Explorer -> Marine Charging
# Assumes AC and DC current
#   - AC = AC 4,29 kr/kWh
#   - AC = DC 5,39 Kr/kWh

tarrif_df["tarrif_types"].unique()

array(['DKK 2.69 per kWh', 'DKK 2,99 per kWh', 'kr 5,39/kWh',
       'kr 4,29 per kWh', 'kr 5,39 / kWh', 'kr 4,29/kWh',
       'EUR 0.55 per kWh', 'kr 5,99/kWh. Over 80%: + kr 1/minutt',
       '5,49 SEK per kWh', '3,39 SEK per kWh', 'kr 3.99 per kWh',
       'kr 3,99 per kWh', 'DC 5,39 kr/kWh + 1kr/min over 80%',
       'AC 4,29 kr/kWh', 'kr 2,5 per kWh',
       'Dynamic. Current price: kr 0,0 per kWh', 'kr 0 per kWh',
       'DC 5,39 Kr/kWh + 1kr/min over 80%', '4,29 kr/kWh + 0 kr/min',
       '4,0 kr/kWh + MVA, hurtiglading Lofoten',
       'kr 2,80 per kWh + MVA, Landstrøm Lofoten',
       'Lading for Brim Explorer'], dtype=object)

In [59]:
# From the analysis we can divide the tarriff column into 7 columns:
# - Price
# - Currency (ej: kr)
# - Measurement (ej: kWh)
# - Has extra tarriff (ej: Over 80%: + kr 1/minutt)
# - Has VAT (4,0 kr/kWh + MVA, hurtiglading Lofoten), default to false
# - VAT Location (ej: hurtiglading Lofoten)

# We can drop Marine Charging
# We can assume prices are dynamically changing.

# Doesn't have a number
capture_group = tarrif_df["tarrif_types"].str.contains(r'^(\d+[.,]?\d*)')
tarrif_df.drop(tarrif_df[capture_group].index, inplace=True)

tarrif_df["price"] = tarrif_df["tarrif_types"].str.contains(r'\d+[.,]?\d*')
tarrif_df["measurement"] = tarrif_df["tarrif_types"].str.contains(r'(kWh|MWh|Wh)')

/tmp/ipykernel_25606/1957759085.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  capture_group = tarrif_df["tarrif_types"].str.contains(r'^(\d+[.,]?\d*)')
/tmp/ipykernel_25606/1957759085.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tarrif_df["measurement"] = tarrif_df["tarrif_types"].str.contains(r'(kWh|MWh|Wh)')


NameError: name 'text' is not defined

In [60]:
for item in tarrif_df["tarrif_types"]:

    properties = {}
    plus_index = item.find("+")

    if plus_index != -1:
        text = item.str[0:plus_index]

        properties["text"] = text
        properties["has_extra"] = True

    else:
        properties["text"] = item
        properties["has_extra"] = False

    print(properties)


{'text': 'DKK 2.69 per kWh', 'has_extra': False}
{'text': 'DKK 2.69 per kWh', 'has_extra': False}
{'text': 'DKK 2.69 per kWh', 'has_extra': False}
{'text': 'DKK 2.69 per kWh', 'has_extra': False}
{'text': 'DKK 2,99 per kWh', 'has_extra': False}
{'text': 'DKK 2,99 per kWh', 'has_extra': False}
{'text': 'DKK 2,99 per kWh', 'has_extra': False}
{'text': 'DKK 2,99 per kWh', 'has_extra': False}
{'text': 'kr 5,39/kWh', 'has_extra': False}
{'text': 'kr 5,39/kWh', 'has_extra': False}
{'text': 'kr 5,39/kWh', 'has_extra': False}
{'text': 'kr 5,39/kWh', 'has_extra': False}
{'text': 'kr 5,39/kWh', 'has_extra': False}
{'text': 'kr 5,39/kWh', 'has_extra': False}
{'text': 'kr 4,29 per kWh', 'has_extra': False}
{'text': 'kr 4,29 per kWh', 'has_extra': False}
{'text': 'kr 4,29 per kWh', 'has_extra': False}
{'text': 'kr 4,29 per kWh', 'has_extra': False}
{'text': 'kr 4,29 per kWh', 'has_extra': False}
{'text': 'kr 4,29 per kWh', 'has_extra': False}
{'text': 'kr 4,29 per kWh', 'has_extra': False}
{'text':

AttributeError: 'str' object has no attribute 'str'